In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
%matplotlib inline


Transfer Learning for Computer Vision Tutorial
==============================================

These two major transfer learning scenarios look as follows:

-  **Finetuning the convnet**: Instead of random initialization, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.
-  **ConvNet as fixed feature extractor**: Here, we will freeze the weights
   for all of the network except that of the final fully connected
   layer. This last fully connected layer is replaced with a new one
   with random weights and only this layer is trained.


In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

Load Data
---------

We will use torchvision and torch.utils.data packages for loading the
data.

The problem we're going to solve today is to train a model to classify
**ants** and **bees**. We have about 120 training images each for ants and bees.
There are 75 validation images for each class. Usually, this is a very
small dataset to generalize upon, if trained from scratch. Since we
are using transfer learning, we should be able to generalize reasonably
well.

This dataset is a very small subset of imagenet.

.. Note ::
   Download the data from
   `here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>`_
   and extract it to the current directory.



In [4]:
# Data augmentation and normalization for training
# Just normalization for validation
image_transforms  = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=45),
        transforms.RandomHorizontalFlip(),
        transforms.Resize(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

dataset = 'microsphere2'
train_directory = os.path.join(dataset, 'train')
valid_directory = os.path.join(dataset, 'valid')

batch_size = 32
num_classes = 173

data = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid'])

}


train_data_size = len(data['train'])
valid_data_size = len(data['valid'])

train_data = DataLoader(data['train'], batch_size=batch_size, shuffle=True)
valid_data = DataLoader(data['valid'], batch_size=batch_size, shuffle=True)

print(train_data_size, valid_data_size)

1038 346


迁移学习
---------


这里使用ResNet-50的预训练模型。


In [5]:
resnet50 = models.resnet50(pretrained=True)

for param in resnet50.parameters():
    param.requires_grad = True

fc_inputs = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(64, 173),
    nn.LogSoftmax(dim = 1)


)
resnet50 = resnet50.to('cuda:0')
loss_func = nn.NLLLoss()
optimizer = optim.AdamW(resnet50.parameters(),lr=1e-4)

Training the model
------------------

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.



In [6]:
def train_and_valid(model, loss_function, optimizer, epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    history = []
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))

        model.train()

        train_loss = 0.0
        train_acc = 0.0
        valid_loss = 0.0
        valid_acc = 0.0

        for i, (inputs, labels) in enumerate(train_data):
            inputs = inputs.to(device)
            labels = labels.to(device)

            #因为这里梯度是累加的，所以每次记得清零
            optimizer.zero_grad()

            outputs = model(inputs)

            loss = loss_function(outputs, labels)

            loss.backward()

            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            train_acc += acc.item() * inputs.size(0)

        with torch.no_grad():
            model.eval()

            for j, (inputs, labels) in enumerate(valid_data):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)

                loss = loss_function(outputs, labels)

                valid_loss += loss.item() * inputs.size(0)

                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                valid_acc += acc.item() * inputs.size(0)

        avg_train_loss = train_loss/train_data_size
        avg_train_acc = train_acc/train_data_size

        avg_valid_loss = valid_loss/valid_data_size
        avg_valid_acc = valid_acc/valid_data_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

        if best_acc < avg_valid_acc:
            best_acc = avg_valid_acc
            best_epoch = epoch + 1

        epoch_end = time.time()

        print("Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
            epoch+1, avg_valid_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start
        ))
        print("Best Accuracy for validation : {:.4f} at epoch {:03d}".format(best_acc, best_epoch))
        dataset1 = 'E:\gmd_data\microsphere2_dataAugmented'
        torch.save(model, dataset1+'/models/'+'model_'+str(epoch+1)+'.pt')
#         torch.save(model, 'models/'+dataset+'_model_'+str(epoch+1)+'.pt')
        if epoch + 1 - 100 >  best_epoch:
            break
    return model, history


In [7]:
num_epochs = 3000
trained_model, history = train_and_valid(resnet50, loss_func, optimizer, num_epochs)
torch.save(history, 'models/'+dataset+'_history.pt')



Epoch: 1/3000
Epoch: 001, Training: Loss: 5.1561, Accuracy: 0.6744%, 
		Validation: Loss: 5.1561, Accuracy: 0.5780%, Time: 35.4223s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 2/3000
Epoch: 002, Training: Loss: 5.1551, Accuracy: 0.3854%, 
		Validation: Loss: 5.1551, Accuracy: 0.5780%, Time: 15.2758s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 3/3000
Epoch: 003, Training: Loss: 5.1536, Accuracy: 0.4817%, 
		Validation: Loss: 5.1536, Accuracy: 0.2890%, Time: 14.7084s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 4/3000
Epoch: 004, Training: Loss: 5.1495, Accuracy: 0.4817%, 
		Validation: Loss: 5.1495, Accuracy: 0.0000%, Time: 14.8849s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 5/3000
Epoch: 005, Training: Loss: 5.1454, Accuracy: 0.7707%, 
		Validation: Loss: 5.1454, Accuracy: 0.5780%, Time: 14.9288s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 6/3000
Epoch: 006, Training: Loss: 5.1313, Accuracy: 1.1561%, 
		Validation: L

Epoch: 46/3000
Epoch: 046, Training: Loss: 4.2497, Accuracy: 2.5048%, 
		Validation: Loss: 4.2497, Accuracy: 2.0231%, Time: 14.8823s
Best Accuracy for validation : 0.0405 at epoch 042
Epoch: 47/3000
Epoch: 047, Training: Loss: 4.1530, Accuracy: 2.2158%, 
		Validation: Loss: 4.1530, Accuracy: 3.1792%, Time: 14.5738s
Best Accuracy for validation : 0.0405 at epoch 042
Epoch: 48/3000
Epoch: 048, Training: Loss: 4.2332, Accuracy: 2.1195%, 
		Validation: Loss: 4.2332, Accuracy: 2.3121%, Time: 15.0372s
Best Accuracy for validation : 0.0405 at epoch 042
Epoch: 49/3000
Epoch: 049, Training: Loss: 4.1673, Accuracy: 1.9268%, 
		Validation: Loss: 4.1673, Accuracy: 2.3121%, Time: 14.8889s
Best Accuracy for validation : 0.0405 at epoch 042
Epoch: 50/3000
Epoch: 050, Training: Loss: 4.4582, Accuracy: 1.8304%, 
		Validation: Loss: 4.4582, Accuracy: 0.8671%, Time: 13.7191s
Best Accuracy for validation : 0.0405 at epoch 042
Epoch: 51/3000
Epoch: 051, Training: Loss: 4.2248, Accuracy: 2.6012%, 
		Validat

Epoch: 91/3000
Epoch: 091, Training: Loss: 4.0966, Accuracy: 4.7206%, 
		Validation: Loss: 4.0966, Accuracy: 5.7803%, Time: 14.3621s
Best Accuracy for validation : 0.0607 at epoch 071
Epoch: 92/3000
Epoch: 092, Training: Loss: 3.8007, Accuracy: 4.4316%, 
		Validation: Loss: 3.8007, Accuracy: 5.4913%, Time: 14.8885s
Best Accuracy for validation : 0.0607 at epoch 071
Epoch: 93/3000
Epoch: 093, Training: Loss: 3.8388, Accuracy: 3.6609%, 
		Validation: Loss: 3.8388, Accuracy: 5.7803%, Time: 14.3505s
Best Accuracy for validation : 0.0607 at epoch 071
Epoch: 94/3000
Epoch: 094, Training: Loss: 3.9534, Accuracy: 3.8536%, 
		Validation: Loss: 3.9534, Accuracy: 4.0462%, Time: 14.1448s
Best Accuracy for validation : 0.0607 at epoch 071
Epoch: 95/3000
Epoch: 095, Training: Loss: 3.8692, Accuracy: 4.6243%, 
		Validation: Loss: 3.8692, Accuracy: 4.3353%, Time: 14.1646s
Best Accuracy for validation : 0.0607 at epoch 071
Epoch: 96/3000
Epoch: 096, Training: Loss: 3.6602, Accuracy: 4.1426%, 
		Validat

Epoch: 136/3000
Epoch: 136, Training: Loss: 4.0099, Accuracy: 4.5279%, 
		Validation: Loss: 4.0099, Accuracy: 5.2023%, Time: 14.3378s
Best Accuracy for validation : 0.1040 at epoch 127
Epoch: 137/3000
Epoch: 137, Training: Loss: 3.4798, Accuracy: 5.1060%, 
		Validation: Loss: 3.4798, Accuracy: 10.4046%, Time: 13.8982s
Best Accuracy for validation : 0.1040 at epoch 127
Epoch: 138/3000
Epoch: 138, Training: Loss: 3.3462, Accuracy: 5.0096%, 
		Validation: Loss: 3.3462, Accuracy: 9.5376%, Time: 12.9670s
Best Accuracy for validation : 0.1040 at epoch 127
Epoch: 139/3000
Epoch: 139, Training: Loss: 3.7181, Accuracy: 5.1060%, 
		Validation: Loss: 3.7181, Accuracy: 8.6705%, Time: 14.1229s
Best Accuracy for validation : 0.1040 at epoch 127
Epoch: 140/3000
Epoch: 140, Training: Loss: 3.5422, Accuracy: 6.6474%, 
		Validation: Loss: 3.5422, Accuracy: 8.0925%, Time: 14.1792s
Best Accuracy for validation : 0.1040 at epoch 127
Epoch: 141/3000
Epoch: 141, Training: Loss: 3.3855, Accuracy: 5.5877%, 
		

Epoch: 181/3000
Epoch: 181, Training: Loss: 3.5726, Accuracy: 10.4046%, 
		Validation: Loss: 3.5726, Accuracy: 6.9364%, Time: 14.1020s
Best Accuracy for validation : 0.1358 at epoch 165
Epoch: 182/3000
Epoch: 182, Training: Loss: 3.2189, Accuracy: 7.4181%, 
		Validation: Loss: 3.2189, Accuracy: 8.9595%, Time: 14.3051s
Best Accuracy for validation : 0.1358 at epoch 165
Epoch: 183/3000
Epoch: 183, Training: Loss: 3.8486, Accuracy: 8.9595%, 
		Validation: Loss: 3.8486, Accuracy: 6.6474%, Time: 13.9925s
Best Accuracy for validation : 0.1358 at epoch 165
Epoch: 184/3000
Epoch: 184, Training: Loss: 3.4365, Accuracy: 8.7669%, 
		Validation: Loss: 3.4365, Accuracy: 4.9133%, Time: 14.0555s
Best Accuracy for validation : 0.1358 at epoch 165
Epoch: 185/3000
Epoch: 185, Training: Loss: 3.1655, Accuracy: 7.5145%, 
		Validation: Loss: 3.1655, Accuracy: 8.9595%, Time: 14.4199s
Best Accuracy for validation : 0.1358 at epoch 165
Epoch: 186/3000
Epoch: 186, Training: Loss: 3.0575, Accuracy: 8.0925%, 
		

...

Epoch: 1809/3000
Epoch: 1809, Training: Loss: 1.7939, Accuracy: 37.1869%, 
		Validation: Loss: 1.7939, Accuracy: 38.1503%, Time: 14.4331s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1810/3000
Epoch: 1810, Training: Loss: 1.5331, Accuracy: 41.3295%, 
		Validation: Loss: 1.5331, Accuracy: 36.7052%, Time: 14.7281s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1811/3000
Epoch: 1811, Training: Loss: 1.4181, Accuracy: 40.5588%, 
		Validation: Loss: 1.4181, Accuracy: 43.9306%, Time: 14.2934s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1812/3000
Epoch: 1812, Training: Loss: 1.3664, Accuracy: 41.6185%, 
		Validation: Loss: 1.3664, Accuracy: 46.2428%, Time: 14.8812s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1813/3000
Epoch: 1813, Training: Loss: 1.4030, Accuracy: 44.0270%, 
		Validation: Loss: 1.4030, Accuracy: 39.0173%, Time: 14.6577s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1814/3000
Epoch: 1814, Training: Loss: 1.32

Epoch: 1853/3000
Epoch: 1853, Training: Loss: 1.4023, Accuracy: 43.1599%, 
		Validation: Loss: 1.4023, Accuracy: 46.5318%, Time: 14.1489s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1854/3000
Epoch: 1854, Training: Loss: 1.3833, Accuracy: 42.2929%, 
		Validation: Loss: 1.3833, Accuracy: 46.5318%, Time: 14.8997s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1855/3000
Epoch: 1855, Training: Loss: 1.5591, Accuracy: 41.8112%, 
		Validation: Loss: 1.5591, Accuracy: 38.7283%, Time: 14.9458s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1856/3000
Epoch: 1856, Training: Loss: 1.6137, Accuracy: 39.5954%, 
		Validation: Loss: 1.6137, Accuracy: 35.2601%, Time: 14.1128s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1857/3000
Epoch: 1857, Training: Loss: 1.5334, Accuracy: 40.4624%, 
		Validation: Loss: 1.5334, Accuracy: 37.5723%, Time: 13.8138s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1858/3000
Epoch: 1858, Training: Loss: 1.48

Epoch: 1897/3000
Epoch: 1897, Training: Loss: 1.3403, Accuracy: 43.3526%, 
		Validation: Loss: 1.3403, Accuracy: 47.6879%, Time: 15.0399s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1898/3000
Epoch: 1898, Training: Loss: 1.3705, Accuracy: 41.9075%, 
		Validation: Loss: 1.3705, Accuracy: 45.3757%, Time: 14.3103s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1899/3000
Epoch: 1899, Training: Loss: 1.8157, Accuracy: 43.2563%, 
		Validation: Loss: 1.8157, Accuracy: 33.8150%, Time: 14.7242s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1900/3000
Epoch: 1900, Training: Loss: 1.4988, Accuracy: 41.5222%, 
		Validation: Loss: 1.4988, Accuracy: 38.1503%, Time: 13.4903s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1901/3000
Epoch: 1901, Training: Loss: 1.4987, Accuracy: 40.8478%, 
		Validation: Loss: 1.4987, Accuracy: 40.4624%, Time: 14.5745s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1902/3000
Epoch: 1902, Training: Loss: 1.47

Epoch: 1941/3000
Epoch: 1941, Training: Loss: 1.5321, Accuracy: 43.8343%, 
		Validation: Loss: 1.5321, Accuracy: 41.9075%, Time: 14.9344s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1942/3000
Epoch: 1942, Training: Loss: 1.3199, Accuracy: 40.0771%, 
		Validation: Loss: 1.3199, Accuracy: 42.7746%, Time: 14.7031s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1943/3000
Epoch: 1943, Training: Loss: 1.3608, Accuracy: 41.3295%, 
		Validation: Loss: 1.3608, Accuracy: 48.2659%, Time: 14.6991s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1944/3000
Epoch: 1944, Training: Loss: 1.7753, Accuracy: 40.7514%, 
		Validation: Loss: 1.7753, Accuracy: 38.7283%, Time: 15.1738s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1945/3000
Epoch: 1945, Training: Loss: 1.4428, Accuracy: 43.3526%, 
		Validation: Loss: 1.4428, Accuracy: 44.2197%, Time: 15.2240s
Best Accuracy for validation : 0.5145 at epoch 1566
Epoch: 1946/3000
Epoch: 1946, Training: Loss: 1.39

KeyboardInterrupt: 

In [ ]:
history = np.array(history)
plt.plot(history[:, 0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0, 1)
plt.savefig(dataset+'_loss_curve.png')
plt.show()

plt.plot(history[:, 2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.savefig(dataset+'_accuracy_curve.png')
plt.show()